In [7]:
!pip install poptorch

ERROR: Could not find a version that satisfies the requirement poptorch (from versions: none)
ERROR: No matching distribution found for poptorch


In [8]:
import torch, torch.nn as nn
import popart, poptorch
import snntorch as snn
import snntorch.functional as SF

ModuleNotFoundError: No module named 'poptorch'

In [ ]:
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

batch_size = 128
data_path='/data/mnist'

# Define a transform
transform = transforms.Compose([
            transforms.Resize((28, 28)),
            transforms.Grayscale(),
            transforms.ToTensor(),
            transforms.Normalize((0,), (1,))])

mnist_train = datasets.MNIST(data_path, train=True, download=True, transform=transform)
mnist_test = datasets.MNIST(data_path, train=False, download=True, transform=transform)

# Train using full precision 32-flt
opts = poptorch.Options()
opts.Precision.halfFloatCasting(poptorch.HalfFloatCastingBehavior.HalfUpcastToFloat)

# Create DataLoaders
train_loader = poptorch.DataLoader(options=opts, dataset=mnist_train, batch_size=batch_size, shuffle=True, num_workers=20)
test_loader = poptorch.DataLoader(options=opts, dataset=mnist_test, batch_size=batch_size, shuffle=True, num_workers=20)

In [ ]:
num_steps = 25
beta = 0.9

In [ ]:
class Model(torch.nn.Module):
def __init__(self):
    super().__init__()

    num_inputs = 784
    num_hidden = 1000
    num_outputs = 10

    self.fc1 = nn.Linear(num_inputs, num_hidden)
    self.lif1 = snn.Leaky(beta=beta)
    self.fc2 = nn.Linear(num_hidden, num_output)
    self.lif2 = snn.Leaky(beta=beta)

    # Cross-Entropy Spike Count Loss
    self.loss_fn = SF.ce_count_loss()

def forward(self, x, labels=None):
    mem1 = self.lif1.init_leaky()
    mem2 = self.lif2.init_leaky()

    spk2_rec = []
    mem2_rec = []

    for step in range(num_steps):
        cur1 = self.fc1(x.view(batch_size,-1))
        spk1, mem1 = self.lif1(cur1, mem1)
        cur2 = self.fc2(spk1)
        spk2, mem2 = self.lif2(cur2, mem2)

        spk2_rec.append(spk2)
        mem2_rec.append(mem2)

    spk2_rec = torch.stack(spk2_rec)
    mem2_rec = torch.stack(mem2_rec)

    if self.training:
        return spk2_rec, poptorch.identity_loss(self.loss_fn(mem2_rec, labels), "none")
    return spk2_rec

In [ ]:
self.fc1 = nn.Linear(num_inputs, num_hidden)
self.lif1 = snn.Leaky(beta=beta)
self.fc2 = nn.Linear(num_hidden, num_output)
self.lif2 = snn.Leaky(beta=beta)

In [ ]:
from snntorch import surrogate

self.lif1 = snn.Leaky(beta=beta, spike_grad = surrogate.fast_sigmoid())

In [ ]:
self.loss_fn = SF.ce_count_loss()

In [ ]:
mem1 = self.lif1.init_leaky()
mem2 = self.lif2.init_leaky()

In [ ]:
for step in range(num_steps):
    cur1 = self.fc1(x.view(batch_size,-1))
    spk1, mem1 = self.lif1(cur1, mem1)
    cur2 = self.fc2(spk1)
    spk2, mem2 = self.lif2(cur2, mem2)

In [ ]:
net = Model()
optimizer = poptorch.optim.Adam(net.parameters(), lr=0.001, betas=(0.9, 0.999))

poptorch_model = poptorch.trainingModel(net, options=opts, optimizer=optimizer)

epochs = 10
for epoch in tqdm(range(epochs), desc="epochs"):
    correct = 0.0

    for i, (data, labels) in enumerate(train_loader):
        output, loss = poptorch_model(data, labels)

        if i % 250 == 0:
            _, pred = output.sum(dim=0).max(1)
            correct = (labels == pred).sum().item()/len(labels)

            # Accuracy on a single batch
            print("Accuracy: ", correct)